# VacationPy

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)
# Convert the temperature from Kelvin to Celsius
city_data_df['Max Temp'] = city_data_df['Max Temp'] - 273.15

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,betio village,1.3580,172.9211,28.02,78,100,6.91,KI,1704708784
1,1,anadyr,64.7500,177.4833,-13.72,85,100,10.00,RU,1704708785
2,2,grytviken,-54.2811,-36.5092,4.43,84,15,2.83,GS,1704708787
3,3,haiku-pauwela,20.9219,-156.3051,24.47,85,0,6.17,US,1704708788
4,4,peterhead,57.5058,-1.7981,-1.78,79,13,1.52,GB,1704708790


# Step 1: Create a map that displays a point for every city in the city_data_df DataFrame. The size of the point should be the humidity in each city.

In [3]:

# Configure the map plot
map_plot_1= city_data_df.hvplot.points("Lng",
                                       "Lat",
                                       geo =True, 
                                       tiles = "OSM",
                                       frame_width=650,
                                       frame_height=500, 
                                       size="Humidity", 
                                       color="City"
                                      )

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

# Step 2: Narrow down the city_data_df DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cond_df =city_data_df.loc[((city_data_df["Max Temp"]<=27.0) & (city_data_df["Max Temp"] >=21.0)) & (city_data_df["Wind Speed"]< 4.5) & (city_data_df["Cloudiness"]==0),:] 


# Drop any rows with null values
ideal_cond_df = ideal_cond_df.dropna()

# Display sample data
ideal_cond_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,hawaiian paradise park,19.5933,-154.9731,21.69,98,0,1.54,US,1704708809
27,27,ha'il,27.5219,41.6907,22.01,43,0,3.09,SA,1704708821
80,80,brisas de zicatela,15.8369,-97.0419,24.03,76,0,1.91,MX,1704708893
96,96,ar ruways,24.1103,52.7306,22.97,50,0,3.59,AE,1704708918
102,102,khada,27.1833,83.8833,23.37,36,0,4.34,IN,1704708925


# Step 3: Create a new DataFrame called hotel_df.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cond_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
17,hawaiian paradise park,US,19.5933,-154.9731,98,
27,ha'il,SA,27.5219,41.6907,43,
80,brisas de zicatela,MX,15.8369,-97.0419,76,
96,ar ruways,AE,24.1103,52.7306,50,
102,khada,IN,27.1833,83.8833,36,


In [9]:
print(geoapify_key)

bc222d5cbbae4fc49d201b169aba3e46


# Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
# categories = "accommodation.hotel"
# limit =20
params = { "categories":"accommodation.hotel",
    "apikey":geoapify_key,
    "limit":20
   }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude= row["Lat"]
    longitude= row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
            print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
hawaiian paradise park - nearest hotel: No hotel found
ha'il - nearest hotel: No hotel found
brisas de zicatela - nearest hotel: No hotel found
ar ruways - nearest hotel: No hotel found
khada - nearest hotel: No hotel found
mandal - nearest hotel: No hotel found
kailua-kona - nearest hotel: No hotel found
sanchor - nearest hotel: No hotel found
jaffa - nearest hotel: No hotel found
tupa - nearest hotel: No hotel found
dipayal - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
17,hawaiian paradise park,US,19.5933,-154.9731,98,No hotel found
27,ha'il,SA,27.5219,41.6907,43,No hotel found
80,brisas de zicatela,MX,15.8369,-97.0419,76,No hotel found
96,ar ruways,AE,24.1103,52.7306,50,No hotel found
102,khada,IN,27.1833,83.8833,36,No hotel found
215,mandal,IN,25.4500,74.5667,42,No hotel found
225,kailua-kona,US,19.6406,-155.9956,81,No hotel found
348,sanchor,IN,24.7553,71.7722,22,No hotel found
373,jaffa,IL,32.0504,34.7522,50,No hotel found
534,tupa,BR,-21.9347,-50.5136,52,No hotel found


# Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=650,
    frame_height=500,
    size="Humidity",
    color="City",
    hover_cols=["Hotel Name","Country"]
    )

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)